In [123]:
import numpy as np
import re

In [124]:
fen_strings = ['rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
               'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1',
               'rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR w KQkq c6 0 2',
               'rnbqkbnr/pp1ppppp/8/2p5/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2']

In [125]:
def fen_to_input(fen):
    piece_translator = {'P' : [ 1, 0, 0, 0, 0, 0],
                        'R' : [ 0, 1, 0, 0, 0, 0],
                        'N' : [ 0, 0, 1, 0, 0, 0],
                        'B' : [ 0, 0, 0, 1, 0, 0],
                        'Q' : [ 0, 0, 0, 0, 1, 0],
                        'K' : [ 0, 0, 0, 0, 0, 1],
                        'p' : [-1, 0, 0, 0, 0, 0],
                        'r' : [ 0,-1, 0, 0, 0, 0],
                        'n' : [ 0, 0,-1, 0, 0, 0],
                        'b' : [ 0, 0, 0,-1, 0, 0],
                        'q' : [ 0, 0, 0, 0,-1, 0],
                        'k' : [ 0, 0, 0, 0, 0,-1],
                        ' ' : [ 0, 0, 0, 0, 0, 0]}

    result = np.zeros((8,64), dtype=np.int8)
    
    board, to_move, castling_rights, en_passant, halfmove, fullmove = fen.split(' ')
    
    ## PREPROCESS
    board = board.replace('/', '')

    for i in range(2, 9):
        board = board.replace(str(i), ' ' * i)

    for i in range(len(board)):
        result[:6,i] = piece_translator[board[i]]
        
    if to_move == 'w':
        result[7, 0] = 1
    elif to_move == 'b':
        result[7, 0] = -1
        
    if not en_passant == '-':
        row = 9 - int(en_passant[1])
        col = ord(en_passant[0]) - 96
        pos = row * col - 1

        result[6, pos] = result[7, 0]
    
    if 'K' in castling_rights:
        result[7,1] = 1

    if 'Q' in castling_rights:
        result[7,2] = 1

    if 'k' in castling_rights:
        result[7,3] = -1

    if 'q' in castling_rights:
        result[7,4] = -1
        
    return result

In [1]:
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD

Using TensorFlow backend.


In [119]:
# Import everything we need.
import numpy as np
import keras
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential

standard_layer = Dense(100,
                       activation = 'relu',
                       weights = [np.random.random((100,100)),np.random.random(100)]
                       )


# Build the model
model = Sequential() # Sequential model:
model.add(Dense(100, activation = 'relu', input_shape=(8,64),
               weights=[np.random.random((64,100)), np.random.random((100,))]))
model.add(standard_layer)
model.add(standard_layer)
model.add(standard_layer)
model.add(standard_layer)
model.add(standard_layer)
model.add(standard_layer)
model.add(standard_layer)
model.add(standard_layer)
model.add(Dense(2, activation = 'softmax'))

In [121]:
model.layers[0].get_weights()[0]

array([[ 0.41868708,  0.11322462,  0.39242667, ...,  0.35829583,
         0.31976575,  0.96836966],
       [ 0.37236232,  0.76908851,  0.11280847, ...,  0.5933333 ,
         0.24942395,  0.44184029],
       [ 0.57407349,  0.64515376,  0.42557988, ...,  0.66215932,
         0.29121712,  0.47021866],
       ..., 
       [ 0.21678397,  0.60049266,  0.63349026, ...,  0.6885711 ,
         0.50743431,  0.80080467],
       [ 0.32676324,  0.27421075,  0.88359195, ...,  0.82012218,
         0.90175802,  0.993819  ],
       [ 0.72929031,  0.12560391,  0.32315949, ...,  0.60887462,
         0.48863217,  0.80949956]], dtype=float32)

In [ ]:
f = fen_to_input(fen_strings[0])
model.predict(np.array([f]))